In [1]:
import importlib
import pandas as pd
import requests
from bs4 import BeautifulSoup
import nbformat
from io import StringIO
import os
import re
import numpy as np

# Scrape basic stat datasets from FantasyPros.com

## Scrape overall scoring data

In [31]:
# List of URLs to scrape for fantasy stats
urls = [
    'https://www.fantasypros.com/nfl/stats/qb.php?scoring=HALF&roster=y',
    'https://www.fantasypros.com/nfl/stats/rb.php?scoring=HALF&roster=y',
    'https://www.fantasypros.com/nfl/stats/wr.php?scoring=HALF&roster=y',
    'https://www.fantasypros.com/nfl/stats/te.php?scoring=HALF&roster=y'
]

# Initialize an empty list to store DataFrames
data_frames = []

for url in urls:
    # Send an HTTP GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the table on the page
        table = soup.find('table')

        table_string = str(table)
        table_io = StringIO(table_string)
        # Read the table into a DataFrame
        df = pd.read_html(table_io)[0]       
        
        # Add a "LOC" column to the DataFrame
        loc = url.split('/')[-1][:2]
        df[("LOC", "POS")] = loc

        data_frames.append(df)
    else:
        print(f"Failed to retrieve data from {url}")

# Merge all DataFrames into one based on the first and second row headers
merged_df = pd.concat(data_frames, ignore_index=True)

# Combine values in column names (headers) and row 0
merged_df.columns = merged_df.columns.map(' '.join)

# Reset the index
merged_df.reset_index(drop=True, inplace=True)

# Rename columns as specified
merged_df = merged_df.rename(columns={"Unnamed: 0_level_0 Rank": "POS RANK", "Unnamed: 1_level_0 Player": "PLAYER", "LOC POS": "POS"})

# # Get the current working directory
# current_directory = os.getcwd()

# # Get the parent directory
# parent_directory = os.path.dirname(current_directory)

# # Change the working directory to the parent directory
# os.chdir(parent_directory)

merged_df.to_csv('datasets/overall_scoring.csv', index=False)

## Scrape snap counts data

In [32]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# List of URLs to scrape for snap counts
snap_count_urls = [
    'https://www.fantasypros.com/nfl/reports/snap-counts/rb.php?show=perc',
    'https://www.fantasypros.com/nfl/reports/snap-counts/wr.php?show=perc',
    'https://www.fantasypros.com/nfl/reports/snap-counts/te.php?show=perc'
]

# Initialize an empty list to store DataFrames for snap counts
snap_count_data_frames = []

for url in snap_count_urls:
    # Send an HTTP GET request to the URL for snap counts
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the table on the page
        table = soup.find('table')

        html_content = str(table)
        df = pd.read_html(StringIO(html_content), header=[0])[0]  
        
        # Add a "POS" column to the DataFrame for snap counts
        pos = url.split('/')[-1][:2]
        df[("POS")] = pos

        snap_count_data_frames.append(df)
    else:
        print(f"Failed to retrieve data from {url} (snap counts)")

# Concatenate (append) all DataFrames for snap counts
snap_count_merged_df = pd.concat(snap_count_data_frames, ignore_index=True)

# If you want to save the data to a CSV file, you can do it like this:
snap_count_merged_df.to_csv('datasets/snap_counts.csv', index=False)


# Scrape weekly historical scoring data

## Generate URLs to scrape

In [2]:
# Get user input for the number of weeks to scrape
num_weeks = int(input("Enter the previous week for data collection: "))

# Define the base URL and the page options
base_url = "https://www.fantasypros.com/nfl/stats/"

# List of page options
pages = ['qb.php', 'wr.php', 'rb.php', 'te.php']

# Initialize the list to store the generated URLs
urls = []

# Generate URLs based on user input for the 2020, 2021, 2022, and 2023 seasons
for page in pages:
    for year in range(2021, 2024):  # Loop through years 2020, 2021, 2022, and 2023
        max_week = 18 if year > 2020 else 17  # Weeks 1-17 for 2020, Weeks 1-18 for 2021 and 2022
        for week in range(1, max_week + 1):
            if year == 2023 and week > num_weeks: # Weeks 1-INPUT for 2023
                break  # Stop generating URLs for 2022 if the desired week is reached
            url = f"{base_url}{page}?year={year}&range=week&week={week}"
            urls.append(url)

# # Print the list of generated URLs
# for url in urls:
#     print(url)



## Perform scrape

In [7]:
final_dataset = pd.DataFrame()

# Iterate through the URLs
for url in urls:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Assuming the data is in a table, you may need to adjust the code based on the actual structure
        table = soup.find('table')

    
        table_string = str(table)
        table_io = StringIO(table_string)
        # Read the table into a DataFrame
        df = pd.read_html(table_io)[0]

        # Add a "LOC" column to the DataFrame
        loc = url.split('/')[-1][:2]
        df[("LOC", "POS")] = loc
        
        # Extract week value from the URL
        week_value = int(url.split('week=')[1])
        # Regular expression pattern to match the year
        pattern = r'year=(\d{4})'

        # Use re.search to find the match
        match = re.search(pattern, url)
        year_value = match.group(1)
        
        # Add a new 'Week' column with the week value
        df['WEEK'] = week_value
        df['YEAR'] = int(year_value)
        
        # Concatenate the DataFrame to the final dataset
        final_dataset = pd.concat([final_dataset, df], ignore_index=True)
    else:
        print(f"Failed to fetch data from URL: {url}")

# Now, final_dataset contains the combined data with a 'Week' column
final_dataset.head(10)

# Combine values in column names (headers) and row 0
final_dataset.columns = final_dataset.columns.map(' '.join)

# Reset the index
final_dataset.reset_index(drop=True, inplace=True)

# Rename columns as specified
final_dataset = final_dataset.rename(columns={"Unnamed: 0_level_0 Rank": "POS RANK", "Unnamed: 1_level_0 Player": "PLAYER", "LOC POS": "POS", "WEEK ": "WEEK", "YEAR ": "YEAR"})

# Create weighted on date column
# Define a custom function to create a date
def create_date(row):
    year = row['YEAR']
    week = row['WEEK']
    day_of_week = 1  # You can choose a specific day of the week

    # Create a date by considering the year, week, and day_of_week
    date = pd.to_datetime(f'{year}-W{week}-{day_of_week}', format='%Y-W%U-%w')

    return date

# Apply the custom function to create the 'Date' column
final_dataset['DATE'] = final_dataset.apply(create_date, axis=1)

# Define your custom weight function, for example, exponential decay
def calculate_weight(row, date_column, decay_factor):
    days_ago = (pd.to_datetime('now') - row[date_column]).days  # Calculate days ago
    weight = np.exp(-decay_factor * days_ago)  # Exponential decay function
    return weight

# Define a decay factor (you can adjust this value based on the desired decay rate)
decay_factor = 0.1  # Adjust as needed

# Apply the weight function to create a 'Weight' column
final_dataset['WEIGHT'] = final_dataset.apply(calculate_weight, args=('DATE', decay_factor), axis=1)

pattern = r'\((.*?)\)'
final_dataset['TEAM'] = final_dataset['PLAYER'].apply(lambda x: re.search(pattern, x).group(1) if re.search(pattern, x) else pd.NA)

/var/folders/qd/rhs7cnfj66l4p8v_yrkw8n4m0000gn/T/ipykernel_72881/1075884743.py:69: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  days_ago = (pd.to_datetime('now') - row[date_column]).days  # Calculate days ago


## Add schedules

In [8]:
import os
import requests
from bs4 import BeautifulSoup
import csv

# URL for the ESPN NFL schedule grid
base_url = "https://www.espn.com/nfl/schedulegrid/_/year/"

# Years for which you want to scrape data
years = [2021, 2022, 2023]
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Directory to save the combined CSV file
save_directory = "datasets"
combined_csv_filename = os.path.join(save_directory, "master_schedule.csv")

# Create the directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Flag to indicate whether to include headers or not
include_headers = True

# Open the file in 'w' mode to overwrite if it already exists
with open(combined_csv_filename, 'w', newline='', encoding='utf-8') as combined_csv_file:
    combined_csv_writer = csv.writer(combined_csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for year in years:
        url = f"{base_url}{year}"

        # Send a GET request to the URL
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the tables on the page
            tables = soup.find_all('table')

            # Process each table
            for index, table in enumerate(tables):
                # Extract data from the table as needed
                rows = table.find_all('tr')

                # Prepare data for writing to the combined CSV file
                data_rows = []
                for row in rows[1:]:
                    data_rows.append([year] + [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])])

                # Include headers only for the first iteration
                if include_headers:
                    combined_csv_writer.writerow([cell.get_text(strip=True) for cell in rows[0].find_all('th')])
                    include_headers = False  # Set to False after writing headers

                combined_csv_writer.writerows(data_rows[1:])  # Exclude the first row
        else:
            print(f"Failed to retrieve data for the year {year}. Status code: {response.status_code}")

print(f"All data combined and saved to {combined_csv_filename}")

import pandas as pd

df = pd.read_csv("datasets/master_schedule.csv", header=None)

# List of new column names
new_column_names = ['YEAR', 'TEAM', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']

# Use the rename method
df.rename(columns=dict(zip(df.columns, new_column_names)), inplace=True)

df = df.replace('@', '', regex=True)

df.to_csv("datasets/master_schedule.csv", index=False)

Year 2021 data appended to datasets/master_schedule.csv

Year 2022 data appended to datasets/master_schedule.csv

Year 2023 data appended to datasets/master_schedule.csv

All data combined and saved to datasets/master_schedule.csv


,YEAR,TEAM,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,2021,ARI,TEN,MIN,JAX,LAR,SF,CLE,HOU,GB,SF,CAR,SEA,BYE,CHI,LAR,DET,IND,DAL,SEA
1,2021,ATL,PHI,TB,NYG,WSH,NYJ,BYE,MIA,CAR,NO,DAL,NE,JAX,TB,CAR,SF,DET,BUF,NO
2,2021,BAL,LV,KC,DET,DEN,IND,LAC,CIN,BYE,MIN,MIA,CHI,CLE,PIT,CLE,GB,CIN,LAR,PIT
3,2021,BUF,PIT,MIA,WSH,HOU,KC,TEN,BYE,MIA,JAX,NYJ,IND,NO,NE,TB,CAR,NE,ATL,NYJ
4,2021,CAR,NYJ,NO,HOU,DAL,PHI,MIN,NYG,ATL,NE,ARI,WSH,MIA,BYE,ATL,BUF,TB,NO,TB
5,2021,CHI,LAR,CIN,CLE,DET,LV,GB,TB,SF,PIT,BYE,BAL,DET,ARI,GB,MIN,SEA,NYG,MIN
6,2021,CIN,MIN,CHI,PIT,JAX,GB,DET,BAL,NYJ,CLE,BYE,LV,PIT,LAC,SF,DEN,BAL,KC,CLE
7,2021,CLE,KC,HOU,CHI,MIN,LAC,ARI,DEN,PIT,CIN,NE,DET,BAL,BYE,BAL,LV,GB,PIT,CIN
8,2021,DAL,TB,LAC,PHI,CAR,NYG,NE,BYE,MIN,DEN,ATL,KC,LV,NO,WSH,NYG,WSH,ARI,PHI
9,2021,DEN,NYG,JAX,NYJ,BAL,PIT,LV,CLE,WSH,DAL,PHI,BYE,LAC,KC,DET,CIN,LV,LAC,KC


## Final Join and Save

In [9]:
weekly_scoring = pd.read_csv("datasets/weekly_scoring.csv")
master_schedule = pd.read_csv("datasets/master_schedule.csv")

dataset_a = weekly_scoring
dataset_b = master_schedule

# Reshape dataset_b to have columns ['week', 'team', 'opponent']
melted_dataset_b = pd.melt(dataset_b, id_vars=['YEAR', 'TEAM'], var_name='WEEK', value_name='OPP')

# Convert 'WEEK' column to integer
melted_dataset_b['WEEK'] = melted_dataset_b['WEEK'].astype(int)

# Merge datasets based on 'week', 'team', and 'year'
merged_dataset = pd.merge(dataset_a, melted_dataset_b, on=['WEEK', 'TEAM', 'YEAR'], how='left')

# Display the merged DataFrame
merged_dataset.to_csv("datasets/weekly_scoring.csv", index=False)

## PRINT DATAFRAME DATA
# Assuming you have a DataFrame named 'df'
# Get the number of rows
num_rows = final_dataset.shape[0]

# Get the number of columns
num_columns = final_dataset.shape[1]

# Calculate the product of rows and columns
rows_times_columns = num_rows * num_columns

# Print the results
print(f"Number of Rows: {num_rows}")
print(f"Number of Columns: {num_columns}")
print(f"Rows times Columns: {rows_times_columns}")

file_path = "datasets/weekly_scoring.csv"  # Replace with the path to your CSV file

# Check if the file exists
if os.path.exists(file_path):
    # Get the size of the file in bytes
    file_size_bytes = os.path.getsize(file_path)

    # Convert bytes to megabytes
    file_size_mb = file_size_bytes / (1024 * 1024)

    print(f"File Size: {file_size_mb:.2f} MB")
else:
    print("File not found.")

Number of Rows: 20647
Number of Columns: 34
Rows times Columns: 701998
File Size: 2.74 MB


# Push Updated Files

In [10]:
# import subprocess

# # List of file paths to push
# file_paths = ["datasets/"]

# # Specify the GitHub repository URL
# repo_url = "https://github.com/jtaylor515/FFanalysis.git"

# # Specify your commit message
# commit_message = "Update dataset files"

# # Git commands to add, commit, and push each file in the list
# for file_path in file_paths:
#     try:
#         subprocess.run(["git", "add", file_path])
#         subprocess.run(["git", "commit", "-m", commit_message])
#         subprocess.run(["git", "push", repo_url])
#         print(f"File {file_path} successfully pushed to the repository.")
#     except Exception as e:
#         print(f"Error: {e}")

[jeff b8b66a8] Update dataset files
 2 files changed, 20745 insertions(+), 39639 deletions(-)
 create mode 100644 datasets/master_schedule.csv
File datasets/ successfully pushed to the repository.


To https://github.com/jtaylor515/FFanalysis.git
   3ca9a75..b8b66a8  jeff -> jeff
